# Hyperparameter Tuning


In [ ]:
# Constants
DATA_PATH = '../data/dataset.xlsx'
MODEL_NAME = "Thammarak/wangchanBERTa-QA-thaiqa_squad"

## Prepare Data

In [ ]:
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

# Read data
df = pd.read_excel(DATA_PATH, sheet_name='Default')

# Function to find answer start
def find_answer_start(context, answer):
    return context.find(answer)

# Add context column
df['context'] = df['Question'] + "          " + df['Answer']

# Rename columns
df.rename(columns={'Answer': 'answers', 'Question': 'question'}, inplace=True)

# Split dataset into train, valid, test
len_test = int(0.2 * len(df))
train, valid, test = df[:-len_test], df[-len_test:-len_test*2], df[-len_test*2:]

# Generate question and article IDs
def generate_ids(dataset, offset):
    return [str(i + offset) for i in range(len(dataset))]

train_question_id = generate_ids(train, 0)
train_article_id = generate_ids(train, 0)
valid_question_id = generate_ids(valid, len(train))
valid_article_id = generate_ids(valid, len(train))
test_question_id = generate_ids(test, len(train) + len(valid))
test_article_id = generate_ids(test, len(train) + len(valid))

# Create Dataset objects
train_set = {
    'question_id': train_question_id,
    'article_id': train_article_id,
    'context': train['context'],
    'question': train['question'],
    'answers': train['answers']
}
valid_set = {
    'question_id': valid_question_id,
    'article_id': valid_article_id,
    'context': valid['context'],
    'question': valid['question'],
    'answers': valid['answers']
}
test_set = {
    'question_id': test_question_id,
    'article_id': test_article_id,
    'context': test['context'],
    'question': test['question'],
    'answers': test['answers']
}

# Create DatasetDict
raw_datasets = DatasetDict({
    'train': Dataset.from_dict(train_set),
    'validation': Dataset.from_dict(valid_set),
    'test': Dataset.from_dict(test_set)
})

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained('airesearch/wangchanberta-base-wiki-20210520-spm-finetune-qa')

# Preprocessing function
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )
    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1

        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            start_positions.append(context_start)
            end_positions.append(context_end)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# Apply preprocessing function
tokenized_raw = raw_datasets.map(
    preprocess_function, batched=True, remove_columns=raw_datasets["train"].column_names
)

## Hyperparameters Tuning: Optuna

In [ ]:
import torch
import logging
import optuna
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer, DefaultDataCollator
from tqdm import tqdm  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME).to(device)

# Data collator
data_collator = DefaultDataCollator()

# Objective function for Optuna
def objective(trial):
    # Define hyperparameters to search over
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 1e-2, log=True)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [4, 8])  # Use categorical suggestion for batch_size

    # Training arguments
    training_args = TrainingArguments(
        output_dir="models-optuna/wangchan_th_wiki_qa",
        overwrite_output_dir=True,
        load_best_model_at_end=True,
        metric_for_best_model='loss',
        save_strategy="epoch",
        greater_is_better=False,
        evaluation_strategy="epoch",
        num_train_epochs=3,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=weight_decay,
    )

    # Prepare datasets
    train_dataset = tokenized_raw['train'].map(lambda examples: {k: torch.tensor(v).to(device) for k, v in examples.items()})
    eval_dataset = tokenized_raw['validation'].map(lambda examples: {k: torch.tensor(v).to(device) for k, v in examples.items()})

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Training loop
    train_dataloader = trainer.get_train_dataloader()
    num_training_steps = len(train_dataloader) * training_args.num_train_epochs
    progress_bar = tqdm(range(num_training_steps), desc="Training")
    for step, batch in enumerate(train_dataloader):
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Log training loss
        if step % 100 == 0:
            print(f"Step {step}/{num_training_steps}, Loss: {loss.item()}")

        progress_bar.update(1)

    progress_bar.close()

    # Evaluate and return loss
    eval_metrics = trainer.evaluate()
    return eval_metrics["eval_loss"]

# Initialize Optuna study
study = optuna.create_study(direction="minimize")

# Optimize hyperparameters
study.optimize(objective, n_trials=30)  

# Get best hyperparameters
best_params = study.best_params

# Print best hyperparameters
print("Best Hyperparameters:", best_params)
